# HardLine Sap Dataset

In [21]:
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.pyplot
import cvxopt
from cvxopt import solvers,matrix
from sklearn.svm import SVC

def lin_Data(size, feature):
    w0 = np.zeros(1)
    w = 0.5 * np.ones(shape=(feature,1))
    std_dev = 0
    data = []
    ex = size/2

# Saperable DataSet
    for i in range(0,size):
        (data.append (np.asarray(np.random.uniform(low=-1,high=1,size=feature),dtype='float32')))

# Non-Saperable DataSet
#     data = np.append((np.random.randn((size/2),feature)+std_dev),(np.random.randn((size/2),feature)-std_dev),axis =0)
    
    data = np.array(data)
    np.random.shuffle(data)
    clabel = []
    for i in range(0,data.shape[0]):
        if(w0 + np.dot(w.T,data[i].T))> 0:
            clabel.append([1.0])
        else:
            clabel.append([-1.0])
    clabel = np.array(clabel)
    Kfold = data.shape[0]/10
    
    testdata = data[-Kfold:,:]
    data = data[0:-Kfold,:]
    test_lab = clabel[-Kfold:,:]
    clabel = clabel[0:-Kfold,:]
# Graph Plot for Generated dataset    
    xp = []
    xn = []
    xpo = []
    ypo = []
    xne = []
    yne = []
    for i in range(0,clabel.shape[0]):
        if clabel[i]>0:
            xp.append(data[i])
        else:
            xn.append(data[i])
    for j in range(0,len(xp)-1):
        xpo.append(xp[j][0])
        ypo.append(xp[j][1])
    for k in range(0,len(xn)-1):
        xne.append(xn[k][0])
        yne.append(xn[k][1])
    plt.plot(xne,yne,'ro')
    plt.plot(xpo,ypo,'bo'),
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Separable Dataset")
    plt.show()
    return data,clabel,testdata,test_lab
size = 200
feature = 2
data, clabel,testdata,test_lab = lin_Data(size,feature)

In [15]:
def alpha_solver(x, y):
    W = 0
    W0 = 0
    var,xshape = x.shape
    p = matrix(np.ndarray.tolist(np.dot(y.T,y) * np.dot(x,x.T)))
    q = cvxopt.base.matrix([-1.0] * var)
    g0 = np.zeros((var, var), float)
    np.fill_diagonal(g0, -1.0)
    g = matrix(np.ndarray.tolist(g0)).T
    h = matrix([0.0] * var)
    a = matrix(np.ndarray.tolist(y))
    b = matrix(0.0)
    Ld = solvers.qp(p,q,g,h,a,b)['x']
    alpha_val = matrix(Ld)

    support_vector = []
    for i in range(len(alpha_val)):
        if alpha_val[i] > 0:
            support_vector.append(x[i])
    support_vector = np.matrix(support_vector)
    
    for j in range(var):
        W += (alpha_val[j] * y[j] * x[j])

    for k in range(len(support_vector)):
        W0 += (y[k] - np.dot(W,x[k]))
    
    W0 = W0 / len(support_vector)
     
    return W0, W, np.array(support_vector)
w0, w, s_vector = alpha_solver(data,clabel)

     pcost       dcost       gap    pres   dres
 0: -1.7659e+02 -4.1175e+02  2e+02  3e-15  2e+00
 1: -3.8284e+02 -3.8794e+02  5e+00  2e-14  1e+00
 2: -2.9185e+04 -2.9189e+04  4e+00  1e-12  1e+00
 3: -2.0814e+08 -2.0814e+08  1e+03  6e-09  1e+00
 4: -2.2416e+08 -2.2416e+08  1e+03  2e-08  1e+00
Terminated (singular KKT matrix).


In [16]:
def predict_class(w0,w,testdata,test_lab,s_vector):
    predict_label = []
    true = 0
    for i in range(0,testdata.shape[0]):
        if(w0 + np.dot(w.T,testdata[i].T))> 0:
            predict_label.append([1.0])
        else:
            predict_label.append([-1.0])
    predict_label = np.array(predict_label)
#     print predict_label, test_lab
    for i in range(0,predict_label.shape[0]):
        if (predict_label[i]==test_lab[i]):
            true = true+1
    
    accuracy = float(true)/len(test_lab)
# Graph of Predicted class value
 
    xp = []
    xn = []
    xpo = []
    ypo = []
    xne = []
    yne = []
    for i in range(0,predict_label.shape[0]):
        if predict_label[i]>0:
            xp.append(testdata[i])
        else:
            xn.append(testdata[i])
    for j in range(0,len(xp)-1):
        xpo.append(xp[j][0])
        ypo.append(xp[j][1])
    for k in range(0,len(xn)-1):
        xne.append(xn[k][0])
        yne.append(xn[k][1])
    spo = []
    sne= []
    
    for s in range(0,len(s_vector)-1):
        spo.append(s_vector[s][0])
        sne.append(s_vector[s][1])
    
       
    testdata = np.array(testdata)
    clf = SVC(kernel='linear')
    clf.fit(testdata,test_lab)
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(-1, 1)
    yy = a * xx+ 0.05  - (clf.intercept_[0]) / w[1]
    margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
    yy_down = yy + a * margin 
    yy_up = yy - a * margin 
    plt.plot(xx, yy_up, 'k--')
    plt.plot(xx, yy, 'k-')
    plt.plot(xx,yy_down,'k--')
    plt.plot(xne,yne,'ro')
    plt.plot(xpo,ypo,'bo')
    plt.scatter(spo,sne,c='yellow',marker='D',s=20)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.title("Predicted value graph")
    plt.title("Hard Margin Prediction")
    plt.axis("tight")
    plt.show()
      
    return predict_label

predict_label = predict_class(w0,w,testdata,test_lab,s_vector)
    

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [17]:
def confusion_matrix(testclass,ypredict):
    
    temp = [-1.0,1.0]
    cm= []
    for i in temp:
        tmp =[0]*len(temp)
        for j in range(len(testclass)):
            if testclass[j] == i and testclass[j] == ypredict[j]:
                tmp[temp.index(i)] += 1
            elif testclass[j] == i and testclass[j] != ypredict[j]:
                tmp[temp.index(ypredict[j])] += 1
        cm.append(tmp)
    cm = np.array(cm)
    print cm
    return cm

    
cm = confusion_matrix(test_lab,predict_label)

[[ 6  0]
 [ 1 13]]


In [18]:
def precisionCall(cm,testclass):
    precesion = np.zeros(2)
    recall = np.zeros(2)
    f1measure = np.zeros(2)
    accuracy = 0
    for i in range(0,2):
        for j in range(0,2):
            precesion[i] += cm[j][i]
            recall[i] += cm[i][j]
            if(i==j):
                accuracy = accuracy + cm[i][j]
        precesion[i] = cm[i][i]/precesion[i]
        recall[i] = cm[i][i]/recall[i]
        f1measure[i] = ((2*precesion[i]*recall[i])/(precesion[i]+recall[i]))
    accuracy = float(accuracy)/testclass.shape[0]
    print " Confusion Matrix:"+ str(cm[0])
    print "\t\t  "+ str(cm[1])
    print "         Precesion:   Recall:    F-1 Measures"
    print "Class-1: " + str(round(precesion[0],3))+"       "+str(round(recall[0],3))+"         "+str(round(f1measure[0],3))
    print "Class 1: " + str(round(precesion[1],3))+"        "+str(round(recall[1],3))+"        "+str(round(f1measure[1],3))
    print "\nAccuracy: "+str(round(accuracy,3)*100)+"%"
    return accuracy,precesion,recall,f1measure


    
accuracy,precesion,recall,f1measure = precisionCall(cm,test_lab)

 Confusion Matrix:[6 0]
		  [ 1 13]
         Precesion:   Recall:    F-1 Measures
Class-1: 0.857       1.0         0.923
Class 1: 1.0        0.929        0.963

Accuracy: 95.0%
